In [2]:
import pandas as pd
import numpy as np
import sys, os
from tqdm.notebook import tqdm
from datetime import timedelta
import matplotlib.pyplot as plt
import pickle

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

In [3]:
chartevents     = pd.read_parquet(icu+'chartevents.parquet')
d_items         = pd.read_parquet(icu+'d_items.parquet')
#inputevents     = pd.read_parquet(icu+'inputevents.parquet')
#procedureevents = pd.read_parquet(icu+'procedureevents.parquet')
#icustays        = pd.read_parquet(icu+'icustays.parquet')

In [5]:
tmp = chartevents[chartevents['itemid'].isin([220739,223900,223901])]

In [6]:
tmp = tmp[['subject_id','hadm_id','stay_id','charttime','itemid','valuenum']]

In [7]:
pivot = tmp.pivot(index=['subject_id','hadm_id','stay_id','charttime'],columns='itemid',values='valuenum')

In [9]:
pivot.reset_index(inplace=True)

In [12]:
pivot['GCS'] = pivot[220739] + pivot[223900] + pivot[223901]

In [18]:
pivot['charttime'] = pd.to_datetime(pivot['charttime'])

In [19]:
pivot.sort_values(by=['subject_id','charttime'],inplace=True)

In [21]:
pivot.GCS.describe()

count    1.615359e+06
mean     1.181600e+01
std      3.714427e+00
min      3.000000e+00
25%      1.000000e+01
50%      1.400000e+01
75%      1.500000e+01
max      1.500000e+01
Name: GCS, dtype: float64

In [20]:
pivot[pivot[220739].isna()]

itemid,subject_id,hadm_id,stay_id,charttime,220739,223900,223901,GCS
423,10002495,24982426,36753294,2141-05-25 00:00:00,NaN,5.0,6.0,NaN
424,10002495,24982426,36753294,2141-05-25 04:00:00,NaN,4.0,6.0,NaN
658,10004235,24181354,34100191,2196-02-26 09:00:00,NaN,NaN,5.0,NaN
680,10004401,22869003,38292466,2144-04-06 04:00:00,NaN,NaN,6.0,NaN
1791,10011365,26712576,37153661,2166-02-15 09:00:00,NaN,1.0,4.0,NaN
...,...,...,...,...,...,...,...,...
1637201,19997473,27787494,32134105,2173-09-21 00:00:00,NaN,5.0,NaN,NaN
1637202,19997473,27787494,32134105,2173-09-21 05:00:00,NaN,5.0,NaN,NaN
1637249,19997752,29452285,34531437,2128-03-07 04:00:00,NaN,5.0,NaN,NaN
1637539,19999068,21606769,30143796,2161-08-30 09:00:00,NaN,4.0,NaN,NaN


In [4]:
labs = d_items[d_items['category']=='Labs'].itemid.unique()

In [7]:
chartevents_labs = chartevents[chartevents['itemid'].isin(labs)]

In [9]:
chartevents_labs = chartevents_labs[['subject_id','hadm_id','stay_id','charttime','itemid','valuenum']]

In [14]:
chartevents_labs.drop_duplicates(inplace=True)

/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_3895/2675119574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chartevents_labs.drop_duplicates(inplace=True)


In [19]:
chartevents_labs['count'] = 1

/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_3895/143980799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chartevents_labs['count'] = 1


In [20]:
chartevents_labs_pivot = chartevents_labs.pivot(index = ['subject_id','hadm_id','stay_id','charttime'],columns = 'itemid',values = 'count')

In [21]:
chartevents_labs_pivot.reset_index(inplace=True)

In [27]:
chartevents_labs_pivot.fillna(0, inplace=True)

In [36]:
chartevents_labs_pivot['sum_lab'] = chartevents_labs_pivot[chartevents_labs_pivot.columns[4:]].sum(axis=1)

In [38]:
chartevents_labs_pivot.sum_lab.describe()

count    1.922695e+06
mean     6.274500e+00
std      7.399284e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      9.000000e+00
max      4.900000e+01
Name: sum_lab, dtype: float64